In [53]:
from pathlib import Path
from collections import Counter
import shutil #.copy2
from functools import lru_cache

from tokenizers import ByteLevelBPETokenizer
from tokenizers.normalizers import NFD, Sequence

In [101]:
with open('utf-refdata/3fb8b25f10e70bc8c4b0246f6ac0ede6.txt', encoding='utf-8') as f:
    print(f.read())

[Distributed to th e C o u n c il and the Members o f t h e L e a g u e ]

C. 524. M. 187. 1924. II.
Geneva, O ctober 30th, 1924.

LEAGUE OF NATIONS

THE
SETTLEMENT OF GREEK REFUGEES
SCHEME FOR AN INTERNATIONAL LOAN

PROTOCOL
Arranged by the League of Nations and signed a t Geneva on September 2 9th , 1 9 2 3 . Amended by the A dditional A c t signed at Geneva on September 19th, 1 9 2 4
W ITH TH E RELEV AN T DOCUMENTS AND PUBLIC STA TEM ENTS, INCLUDING T H E STA TU TES
OF T H E REFU G EE SETTLEM EN T COMMISSION

LEAGUE OF NATIONS
RRATUM AND A D D EN D U M TO D O C U M E N T C. 524. M. 187.1924. II
The Settlement of Greek Refugees.
Page 6 . Heading I I I . -- The beginning of the second paragraph should read as follows: "As will be seen from Document X II, the property so transferred, and to be trans
ferred to, (2) and held with a clean title by, the Refugee Settlement Commission", etc.
Anote should be added a t the bottom of page 6 as follows : (2). "The position as to the delivery of 

In [102]:
paths = [str(x) for x in Path(".").glob("utf-refdata/*.txt") if x.stat().st_size > 0]

In [103]:
%%time 

normalizer = Sequence([NFD()])

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.normalizer = normalizer

# Customize training
tokenizer.train(files=paths, vocab_size=50_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 22h 16min 35s, sys: 6h 4min 44s, total: 1d 4h 21min 19s
Wall time: 59min 2s


In [160]:
tokenizer.save("wfctokenizer/tokenizer.json")
tokenizer.save_model('wfctokenizer/')

['wfctokenizer/vocab.json', 'wfctokenizer/merges.txt']

---

In [106]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./wfctokenizer/vocab.json",
    "./wfctokenizer/merges.txt",
)

In [111]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [113]:
tokenizer.encode("the hindwings are brown with a grey line").tokens

['<s>',
 'the',
 'Ġhind',
 'w',
 'ings',
 'Ġare',
 'Ġbrown',
 'Ġwith',
 'Ġa',
 'Ġgrey',
 'Ġline',
 '</s>']

---

### train a language model

In [115]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [123]:
from datasets import load_dataset

In [137]:
with open('ls_refdata', 'r') as f:
    reffiles = [*map(lambda name: 'utf-refdata/'+name[:-1], f.readlines())]
    (reffiles)
        
dataset = load_dataset('text', 
                       data_files=reffiles[:10])

Using custom data configuration default


Dataset text downloaded and prepared to /home/as9kc/.cache/huggingface/datasets/text/default-494b1f3069319efd/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab. Subsequent calls will reuse this data.


In [153]:
dataset['train'][8]

{'text': ''}

In [131]:
data['train'][0]

{'Unnamed: 0': 0,
 'url': 'https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/20070025021_2007024501.pdf',
 'context': 'A lunar sortie (or lunar sortie mission ) is a human spaceflight mission to the Moon.',
 'id': 2,
 'refuted': 'In contrast with lunar outpost missions, lunar sorties will be of relatively longer duration.',
 'claim': 'In contrast with lunar outpost missions, lunar sorties will be of relatively brief duration.',
 'url_md5': 'c3ad50ad22cb3f06be1bec23492f6053.txt'}